In [13]:
import os
import numpy as np
import cv2
from tqdm import tqdm

In [14]:
DATADIR = "./data/"
CATEGORIES = ["Cat", "Dog"]
IMG_SIZE = 100

In [15]:
training_data = []

In [16]:
MAX_IMAGES_PER_CLASS = 5000  # ou 500, por exemplo

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category.lower())
        class_num = CATEGORIES.index(category)
        count = 0
        for img in tqdm(os.listdir(path)):
            if count >= MAX_IMAGES_PER_CLASS:
                break
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
                count += 1
            except Exception:
                pass


In [17]:
create_training_data()



 40%|████      | 5001/12501 [01:16<01:54, 65.61it/s] 


In [18]:
import random

random.shuffle(training_data)

x = []
y = []

for features, label in training_data:
    x.append(features)
    y.append(label)

x = np.array(x).reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255.0
y = np.array(y)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(1, activation='sigmoid')) 

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [20]:
model.fit(x, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 53s 176ms/step - loss: 0.6797 - accuracy: 0.5824 - val_loss: 0.6574 - val_accuracy: 0.6285
Epoch 2/10
250/250 [==============================] - 42s 170ms/step - loss: 0.6075 - accuracy: 0.6721 - val_loss: 0.5934 - val_accuracy: 0.6815
Epoch 3/10
250/250 [==============================] - 45s 179ms/step - loss: 0.5333 - accuracy: 0.7331 - val_loss: 0.5390 - val_accuracy: 0.7260
Epoch 4/10
250/250 [==============================] - 44s 175ms/step - loss: 0.4667 - accuracy: 0.7830 - val_loss: 0.5307 - val_accuracy: 0.7330
Epoch 5/10
250/250 [==============================] - 43s 174ms/step - loss: 0.4021 - accuracy: 0.8170 - val_loss: 0.6250 - val_accuracy: 0.7180
Epoch 6/10
250/250 [==============================] - 45s 179ms/step - loss: 0.3280 - accuracy: 0.8583 - val_loss: 0.5467 - val_accuracy: 0.7515
Epoch 7/10
250/250 [==============================] - 43s 173ms/step - loss: 0.2745 - accuracy: 0.8854 - val_loss: 0.6286 - val_ac

In [21]:
import matplotlib.pyplot as plt

def prepare(filepath):
    img_array = cv2.imread(filepath)
    resized = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) / 255.0
    return resized.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

prediction = model.predict(prepare("data/Cat/0.jpg"))
print("Dog" if prediction[0][0] > 0.5 else "Cat")

1/1 [==============================] - 0s 104ms/step
Cat


In [22]:
model.save("modelo_caes_gatos.keras")

In [23]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageOps
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# --- CONFIGURAÇÕES ---
IMG_SIZE = 100
MODEL_PATH = "modelo_caes_gatos.keras"

# --- Carregar modelo ---
model = load_model(MODEL_PATH)

# --- Função de Previsão ---
def prepare_image(filepath):
    img = cv2.imread(filepath)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    return img.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

def predict_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Exibir imagem
        img = Image.open(file_path).convert("RGB")
        img = ImageOps.fit(img, (250, 250), Image.Resampling.LANCZOS)
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk

        # Prever
        prediction = model.predict(prepare_image(file_path))
        if prediction[0][0] > 0.5:
            result = "🐶 É um cachorro!"
            color = "#d0f0c0"
        else:
            result = "🐱 É um gato!"
            color = "#f5c8c8"

        result_label.config(text=result, bg=color)

# --- Construção da Interface ---
window = tk.Tk()
window.title("Classificador Cão vs Gato")
window.geometry("350x450")
window.configure(bg="#f0f0f0")

title_label = tk.Label(window, text="Classificador de Cães e Gatos",
                       font=("Arial", 14, "bold"), bg="#f0f0f0", fg="#333")
title_label.pack(pady=10)

btn = tk.Button(window, text="Escolher Imagem 🖼️",
                command=predict_image, font=("Arial", 12), bg="#4caf50", fg="white",
                activebackground="#45a049", padx=10, pady=5, relief="raised")
btn.pack(pady=10)

image_label = tk.Label(window, bg="#dddddd", width=250, height=250, bd=2, relief="groove")
image_label.pack(pady=10)

result_label = tk.Label(window, text="Resultado aparecerá aqui",
                        font=("Arial", 14, "bold"), bg="#ffffff", fg="#000000", bd=2,
                        relief="ridge", width=25, height=2)
result_label.pack(pady=20)

window.mainloop()


1/1 [==============================] - 0s 21ms/step
